<a href="https://colab.research.google.com/github/ruiding-code/QnA/blob/master/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAIS 202 Deliverable 2

https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1174/reports/2761899.pdf

https://pathmind.com/wiki/word2vec (word2vec for data preprocessing)

In [161]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


In [0]:
import os
os.chdir(r"/content/drive/My Drive/McGill/Winter 2020/MAIS 202 Bootcamp/Final Project")

In [163]:
!pwd

/content/drive/My Drive/McGill/Winter 2020/MAIS 202 Bootcamp/Final Project


In [0]:
import csv
import json


In [0]:

datafile = open("train-v2.0.json", "r")


In [0]:
file_string = datafile.read()

In [0]:
file_dict = json.loads(file_string)

In [168]:
file_dict['version']

'v2.0'

In [0]:
data_list = file_dict["data"]
# list of dicts. in each dict: "title": str, "paragraphs": list

The following code block is used to test out the features of the dataset and are not pertinent to data preprocessing or modelling

In [170]:
articles = list()
for article in data_list:
  articles.append(article["title"])
articles = np.array(articles)

print(articles[2])

print(data_list[2]["paragraphs"][0]["context"])

# def find_article(title):
#   return "https://en.wikipedia.org/wiki/" + str(title)



# print(find_article_index("Sino-Tibetan_relations_during_the_Ming_dynasty"))

beyonce = data_list[0]
beyonce_paragraphs = beyonce["paragraphs"]
# list: each list is associated to a given passage of the article
# list of dicts. in each dict: "qas": list of dicts, "context": str


print(beyonce_paragraphs[0]["context"])
print(beyonce_paragraphs[0]["qas"][10:16])



Sino-Tibetan_relations_during_the_Ming_dynasty
The exact nature of relations between Tibet and the Ming dynasty of China (1368–1644) is unclear. Analysis of the relationship is further complicated by modern political conflicts and the application of Westphalian sovereignty to a time when the concept did not exist. Some Mainland Chinese scholars, such as Wang Jiawei and Nyima Gyaincain, assert that the Ming dynasty had unquestioned sovereignty over Tibet, pointing to the Ming court's issuing of various titles to Tibetan leaders, Tibetans' full acceptance of these titles, and a renewal process for successors of these titles that involved traveling to the Ming capital. Scholars within China also argue that Tibet has been an integral part of China since the 13th century and that it was thus a part of the Ming Empire. But most scholars outside China, such as Turrell V. Wylie, Melvin C. Goldstein, and Helmut Hoffman, say that the relationship was one of suzerainty, that Ming titles were only

In [171]:
print(len(data_list[21]["paragraphs"]))

# test out model on paragraphs from 25 first articles in the dataset



# data_list = [article, ..., article]
# article = {"title": str, "paragraphs": [segment, ..., segment]}
# segment = {"qas": [qna-set, ..., qna-set], "context": str}
# qna-set = {"question": str, "id": str, "answers": [{"text": str, "answer_start": int}], "is_impossible": False}

print(len(data_list))

21
442


In [172]:
# division of data: inputs vs. expected output

# need to get a list of questions (X), associated context, and a list of the corresponding start and end answer indices (Y)

questions = [[[qnaset["question"] for qnaset in segment["qas"]] for segment in article["paragraphs"]] for article in data_list]
print(questions[1][0][0]) 
# questions[article_index][segment_index][question_index]

contexts = [[segment["context"] for segment in article["paragraphs"]] for article in data_list]
print(contexts[1][0])
fred_chop = contexts[1][0]
# contexts[article_index][segment_index]

answer_indices = [[[[[answer["answer_start"], answer["answer_start"] + len(answer["text"])] for answer in qnaset["answers"]] for qnaset in segment["qas"]] for segment in article["paragraphs"]] for article in data_list]
print(answer_indices[1][0][0][0])
fred_answ = answer_indices[1][0][0][0]
# answer_indices[article_index][segment_index][question_index][0][start0_or_end1]
print(fred_chop[fred_answ[0]:fred_answ[1]])



What was Frédéric's nationalities?
Frédéric François Chopin (/ˈʃoʊpæn/; French pronunciation: ​[fʁe.de.ʁik fʁɑ̃.swa ʃɔ.pɛ̃]; 22 February or 1 March 1810 – 17 October 1849), born Fryderyk Franciszek Chopin,[n 1] was a Polish and French (by citizenship and birth of father) composer and a virtuoso pianist of the Romantic era, who wrote primarily for the solo piano. He gained and has maintained renown worldwide as one of the leading musicians of his era, whose "poetic genius was based on a professional technique that was without equal in his generation." Chopin was born in what was then the Duchy of Warsaw, and grew up in Warsaw, which after 1815 became part of Congress Poland. A child prodigy, he completed his musical education and composed his earlier works in Warsaw before leaving Poland at the age of 20, less than a month before the outbreak of the November 1830 Uprising.
[182, 199]
Polish and French


word2vec and GloVe do not take the context into consideration


BERT takes context into consideration when encoding words and sentences into vectorized form


https://towardsdatascience.com/bert-for-dummies-step-by-step-tutorial-fb90890ffe03

In [173]:
# data preprocessing
# here we want to vectorize both the inputs (encoder) and decode the output to obtain a string answer

# first install pretrained BERT from PyTorch

# install
!pip install pytorch-pretrained-bert pytorch-nlp

# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# specify GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)



'Tesla T4'

Need to tokenize input data into BERT

[CLS] [SEP]

In [174]:
# add tokens to input questions and input contexts

questions = [[["[CLS] " + qnaset["question"].rstrip("?") + " [SEP]" for qnaset in segment["qas"]] for segment in article["paragraphs"]] for article in data_list]
print(questions[0])

# slitting context string by sentence
contexts = [[segment["context"].split(".") for segment in article["paragraphs"]] for article in data_list]
print(contexts[0][3])

for i in range(len(contexts)):
  for j in range(len(contexts[i])):
    context = "[CLS] "
    for k in range(len(contexts[i][j])):
      context += contexts[i][j][k]
      context += " [SEP]"
    contexts[i][j] = context


print(contexts[0])


[['[CLS] When did Beyonce start becoming popular [SEP]', '[CLS] What areas did Beyonce compete in when she was growing up [SEP]', "[CLS] When did Beyonce leave Destiny's Child and become a solo singer [SEP]", '[CLS] In what city and state did Beyonce  grow up?  [SEP]', '[CLS] In which decade did Beyonce become famous [SEP]', '[CLS] In what R&B group was she the lead singer [SEP]', '[CLS] What album made her a worldwide known artist [SEP]', "[CLS] Who managed the Destiny's Child group [SEP]", '[CLS] When did Beyoncé rise to fame [SEP]', "[CLS] What role did Beyoncé have in Destiny's Child [SEP]", '[CLS] What was the first album Beyoncé released as a solo artist [SEP]', '[CLS] When did Beyoncé release Dangerously in Love [SEP]', '[CLS] How many Grammy awards did Beyoncé win for her first solo album [SEP]', "[CLS] What was Beyoncé's role in Destiny's Child [SEP]", "[CLS] What was the name of Beyoncé's first solo album [SEP]"], ['[CLS] After her second solo album, what other entertainment 

In [175]:
# tokenize input questions and contexts with pretrained BERT tokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_questions = [[[tokenizer.tokenize(question) for question in segment] for segment in article] for article in questions]

# [[[qnaset["question"] for qnaset in segment["qas"]] for segment in article["paragraphs"]] for article in data_list]
print(tokenized_questions[0])

tokenized_contexts = [[tokenizer.tokenize(context) for context in article] for article in contexts]
print(tokenized_contexts[1][1])

[[['[CLS]', 'when', 'did', 'beyonce', 'start', 'becoming', 'popular', '[SEP]'], ['[CLS]', 'what', 'areas', 'did', 'beyonce', 'compete', 'in', 'when', 'she', 'was', 'growing', 'up', '[SEP]'], ['[CLS]', 'when', 'did', 'beyonce', 'leave', 'destiny', "'", 's', 'child', 'and', 'become', 'a', 'solo', 'singer', '[SEP]'], ['[CLS]', 'in', 'what', 'city', 'and', 'state', 'did', 'beyonce', 'grow', 'up', '?', '[SEP]'], ['[CLS]', 'in', 'which', 'decade', 'did', 'beyonce', 'become', 'famous', '[SEP]'], ['[CLS]', 'in', 'what', 'r', '&', 'b', 'group', 'was', 'she', 'the', 'lead', 'singer', '[SEP]'], ['[CLS]', 'what', 'album', 'made', 'her', 'a', 'worldwide', 'known', 'artist', '[SEP]'], ['[CLS]', 'who', 'managed', 'the', 'destiny', "'", 's', 'child', 'group', '[SEP]'], ['[CLS]', 'when', 'did', 'beyonce', 'rise', 'to', 'fame', '[SEP]'], ['[CLS]', 'what', 'role', 'did', 'beyonce', 'have', 'in', 'destiny', "'", 's', 'child', '[SEP]'], ['[CLS]', 'what', 'was', 'the', 'first', 'album', 'beyonce', 'released

Next we need to vectorize the tokenized data

In [176]:

# questions
# Set the maximum sequence length. 
MAX_LEN = 500
# Pad our input tokens
question_ids = pad_sequences([[[tokenizer.convert_tokens_to_ids(question) for question in segment] for segment in article] for article in tokenized_questions],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
question_ids = [tokenizer.convert_tokens_to_ids(token) for token in segment for segment in article for article in tokenized_questions]
question_ids = pad_sequences(question_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


# contexts
# Set the maximum sequence length. 
MAX_LEN = 500
# Pad our input tokens
context_ids = pad_sequences([[tokenizer.convert_tokens_to_ids(context) for context in article] for article in tokenized_contexts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
context_ids = [tokenizer.convert_tokens_to_ids(token) for token in article for article in tokenized_contexts]
context_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")



ValueError: ignored

In [0]:
# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in question_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

# Create attention masks
attention_masksc = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in context_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masksc.append(seq_mask)

In [110]:
def train_test_split(X1, X2, y, train_w):
  split = int(train_w * len(X))
  X1_train = X1[:split]
  X1_test = X1[split:]
  X2_train = X2[:split]
  X2_test = X2[split:]
  y_train = y[:split]
  y_test = y[split:]
  return X1_train, X1_test, X2_train, X2_test, y_train, y_test

q_train, q_test, c_train, c_test, a_train, a_test = train_test_split(question_ids, context_ids, answer_indices, 0.8)
  

NameError: ignored